In [6]:
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)

/home/bruno/Desktop/coarse-discourse-validation


## Annotations download from paper's repo

In [7]:
import requests

url = 'https://raw.githubusercontent.com/google-research-datasets/coarse-discourse/master/coarse_discourse_dataset.json'
filename = os.path.join(PROJ_ROOT,'data', 'raw', 'coarse_discourse_dataset.json')

with open(filename, 'wb') as file:
    response = requests.get(url)
    file.write(response.content)